## Kelley hu4d5-5 VH-V102Y error analysis

In this notebook we will use a random forest model to find the most energetically influential degrees of freedom for the VH-V102Y TI production run. Next, we will compare the sampling of these DOF during TI production to a free energy profile derived from end state GaMD sampling. We will attempt to  correct any inaccurate sampling in the TI data and find the estimated ddG before and after the correction. 

In [2]:
import os
os.chdir("..")
from common_functions import *

### Ingesting initial TI data

In [3]:
os.chdir("./TI_data/VH-V102Y/")
geom_dvdls = pd.read_csv("V102Y_bound.csv")
geom_dvdls_ub = pd.read_csv("V102Y_unbound.csv")

### Original ddG estimate:

In [5]:
orig_dG_bd = geom_dvdls.groupby("Lambda").mean().sum()["weight_dvdl"]
orig_dG_ubd = geom_dvdls_ub.groupby("Lambda").mean().sum()["weight_dvdl"]

empirical_value = 0.00
orig_error = abs((orig_dG_bd - orig_dG_ubd) - empirical_value)

print("Original ddG estimate: ")
print(f"{round(orig_dG_bd - orig_dG_ubd, 10)} kcal/mol ")
print()
print("Original ddG error: ")
print(f"{round(orig_error, 10)} kcal/mol")

Original ddG estimate: 
-0.0534874347 kcal/mol 

Original ddG error: 
0.0534874347 kcal/mol


### Removing correlated variables among candidate degrees of freedom

This is important because we want to limit the noise in our model training. See our methods/supplemental methods section for our process to choose the input features.

In [6]:
X = geom_dvdls.drop([
    "#Frame", "weight_dvdl", "dvdl", "Run", "Lambda", 
], axis=1)

Y = geom_dvdls["weight_dvdl"]

X_scl = pd.DataFrame(StandardScaler().fit_transform(X))
X_scl.columns = X.columns


### Checking to see if there is any cross-correlations within the independent variables

In [7]:
absCorr = abs(X_scl.corr())
for i in absCorr.columns:
    for j in absCorr.index:
        cor = absCorr.loc[i, j]
        if abs(cor) > 0.5 and i != j:
            print(i, j)
            print(cor)
            

### Using random forest model to identify the most energetically influential degrees of freedom

We run our model 25 times, then sort the results by the mean of feature importance across the 25 iterations. For this particular perturbation, the model $R^2$ between the geometric DOF (nearby side chain rotamers or interatomic distances) and the energetic DV/DL was not strong enough for us to check the sampling of these degrees of freedom. 

In [15]:
rfeDefault = RFE(estimator=DecisionTreeRegressor(max_depth=5, random_state=42), n_features_to_select=0.75, step=0.05)
rfDefault = RandomForestRegressor(
    max_depth=10, n_estimators=200, oob_score=True, max_features=0.6, min_samples_leaf = 7, min_samples_split=14, random_state=42
)

pipelineDefault_rf = Pipeline([
    ('feature_scaling', StandardScaler()),
    ('feature_selection', rfeDefault),
    ('regression_model', rfDefault)
])


imps = benchmark_model(pipelineDefault_rf, X_scl, Y, geom_dvdls["Lambda"])
imps[["Mean", "Median"]].sort_values(by="Mean", ascending=False)[:15]

Avg. training r2: 
0.5058
Training r2 std dev: 
0.0009
Avg. test r2: 
0.338
Testing r2 std dev: 
0.006


,Mean,Median
Y570_chi1,0.184328,0.184445
L353_chi2,0.097355,0.107825
v351_y570,0.075625,0.075652
V458_chi1,0.059823,0.032527
L353_chi1,0.051432,0.053720
V458_chi2,0.051175,0.016699
d127_y570,0.046692,0.046669
V351_chi1,0.045827,0.045879
D457_chi2,0.045421,0.029274
M456_chi3,0.041685,0.012929
